# Home work

2010-12-01.csv 와 2010-12-02.csv 두 파일이 있습니다. <br>
두 파일을 불러와 준다음에, 우선 하나로 합칩니다. <br>
다음은 앞서 실습 때 진행했던 코드를 최소한 7개를 사용해서 하나의 정리된 데이터 프레임을 만들어 주세요. (where, count, groupby, descending, map 뭐든 상관 없습니다) <br>
그리고 스샷찍어서 제출해주시면됩니다.

In [20]:
# 데이터 불러오기
df1 = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("file:///home/ubuntu/ybigta/Ybigta-Dataframe-Dataset-/2010-12-01.csv")\
  .coalesce(5)

# 데이터 불러오기
df2 = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("file:///home/ubuntu/ybigta/Ybigta-Dataframe-Dataset-/2010-12-02.csv")\
  .coalesce(5)

In [21]:
df1.schema

StructType(List(StructField(InvoiceNo,StringType,true),StructField(StockCode,StringType,true),StructField(Description,StringType,true),StructField(Quantity,IntegerType,true),StructField(InvoiceDate,TimestampType,true),StructField(UnitPrice,DoubleType,true),StructField(CustomerID,DoubleType,true),StructField(Country,StringType,true)))

In [22]:
df1.columns

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country']

In [23]:
df1.show(4)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 4 rows



In [37]:
# 데이터를 한 곳에 모아둔다
df1.collect()
df2.collect()

[Row(InvoiceNo='536598', StockCode='21421', Description='PORCELAIN ROSE LARGE ', Quantity=12, InvoiceDate=datetime.datetime(2010, 12, 2, 7, 48), UnitPrice=1.25, CustomerID=13090.0, Country='United Kingdom'),
 Row(InvoiceNo='536598', StockCode='21422', Description='PORCELAIN ROSE SMALL', Quantity=16, InvoiceDate=datetime.datetime(2010, 12, 2, 7, 48), UnitPrice=0.85, CustomerID=13090.0, Country='United Kingdom'),
 Row(InvoiceNo='536598', StockCode='22178', Description='VICTORIAN GLASS HANGING T-LIGHT', Quantity=24, InvoiceDate=datetime.datetime(2010, 12, 2, 7, 48), UnitPrice=1.25, CustomerID=13090.0, Country='United Kingdom'),
 Row(InvoiceNo='536598', StockCode='22617', Description='BAKING SET SPACEBOY DESIGN', Quantity=24, InvoiceDate=datetime.datetime(2010, 12, 2, 7, 48), UnitPrice=4.25, CustomerID=13090.0, Country='United Kingdom'),
 Row(InvoiceNo='536599', StockCode='22968', Description='ROSE COTTAGE KEEPSAKE BOX ', Quantity=8, InvoiceDate=datetime.datetime(2010, 12, 2, 7, 49), UnitP

# 1. 데이터 프레임 합치기(Union)


In [60]:
total_df = df1.union(df2)

# 2. 고유 값으로 사용가능한 column 조합 찾기

In [61]:
## 칼럼 가능한 모든 조합 구하기
from itertools import combinations 
from tqdm import tqdm


def find_candidate(df):
    candidate_key = [] # 후보키 리스트
    total_df_count = df.count() # 전체 데이터셋 크기
    
    for i in tqdm(range(1, len(df.columns)+1)): 
        columns = list(combinations(df.columns, i))
        for cols in columns:
            new_df = df.dropDuplicates(cols)
            if new_df.count() == total_df_count:
                candidate_key.append(cols)
                
    return candidate_key


total_df_candidate = find_candidate(total_df)
       
        



100%|██████████| 8/8 [01:59<00:00, 14.92s/it]


In [62]:
# 아쉽게도 고유키는 아무것도 없다!
print(total_df_candidate)

[]


# 3. null 값 체크

In [65]:
total_df.columns

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country']

In [82]:
for col in total_df.columns:
    print("columns %s null values : %d"%(col, total_df.filter((total_df[col].isNull())).count()))

columns InvoiceNo null values : 0
columns StockCode null values : 0
columns Description null values : 12
columns Quantity null values : 0
columns InvoiceDate null values : 0
columns UnitPrice null values : 0
columns CustomerID null values : 1205
columns Country null values : 0


In [80]:
total_df.filter((total_df["Description"].isNull())).show()

+---------+---------+-----------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+-----------+--------+-------------------+---------+----------+--------------+
|   536414|    22139|       null|      56|2010-12-01 11:52:00|      0.0|      null|United Kingdom|
|   536545|    21134|       null|       1|2010-12-01 14:32:00|      0.0|      null|United Kingdom|
|   536546|    22145|       null|       1|2010-12-01 14:33:00|      0.0|      null|United Kingdom|
|   536547|    37509|       null|       1|2010-12-01 14:33:00|      0.0|      null|United Kingdom|
|   536549|   85226A|       null|       1|2010-12-01 14:34:00|      0.0|      null|United Kingdom|
|   536550|    85044|       null|       1|2010-12-01 14:34:00|      0.0|      null|United Kingdom|
|   536552|    20950|       null|       1|2010-12-01 14:34:00|      0.0|      null|United Kingdom|
|   536553